<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/3d1d_advdiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2025-08-06 14:16:12--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-08-06 14:16:12 (71.9 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/ec24876d/releases/pybi

In [ ]:
# @title Convergence test: 3D-FEM,1D-dG time-dependent advection-diffusion

from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

T    = 1.0
#dt   = 0.1
#tpts = int(T/dt)
#inv_dt = Constant(1.0/dt)

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)
  Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Lambda_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Lambda_boundaries, 2)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Velocity terms ===
  q1 = Constant(1.0)
  q3 = as_vector((0,0,q1))

  # === Initial concentration ===
  cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  #dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(50.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Upwind ===
  #u1_up = u1('+')
  #v1_up = v1('-')
  #q3_con = Constant((0,0,q1))
  #q1_up = (dot(q3_con, n) + abs(dot(q3_con, n))) / 2.0
  q1_up = (q1 + abs(q1))/2.0 # works best, gets okay convergence - rates are off
#%BR this is silly: q1=1, so q1_up = q1 = 1

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    cinlet.t = t
    # === Exact solutions ===

    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2])*t + (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pi*t*q*cos(pi*x[2]) :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2]))*t + (kappa/(1+kappa))*(sin(pi*x[2])+2 + (kappa/(1+kappa))*pi*t*q*cos(pi*x[2]))',
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t,
                    q = q1)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)'  # coupling
        ' + area * pi * t * q * cos(pi*x[2])',                # advection
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t,
        q=q1
    )

    # === Checks for force terms ===
    #f3_func = interpolate(f3, V3)
    #f1_func = interpolate(f1, V1)
    #File(f"f3_step{step:03d}.pvd") << f3_func
    #File(f"f1_step{step:03d}.pvd") << f1_func

    # === Variational Forms ===
    a00 = inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda - inner(dot(q3,grad(u3)),v3) *dxOmega
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
        + D_area * inner(grad(u1), grad(v1)) * dxLambda \
        - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
        - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
        + (alpha/avg(h)) * jump(u1) * jump(v1) * dS \
        + kappa * D_perimeter * u1 * v1 * dxLambda \
        - D_area * q1 * u1 * v1.dx(2) * dxLambda \ #%BR: what is this term? we cannot write grad(u1)?
        + D_area * jump(v1) * (q1_up * u1('+') - q1_up * u1('-')) * dS + D_area * q1_up * u1 * v1 * dsLambda(2)
        #%BR: + D_area * jump(v1) * (q1_up * u1('+') - q1_up * u1('-')) * dS + D_area * q1 * u1 * v1 * dsLambda(2)

    #% BR: question: dS is only on interior nodes? are we sure?
    #% BR: what does this term (q1_up * u1('+') - q1_up * u1('-')) do?



    L0 = inner(f3, v3) * dxOmega + inv_dt * (inner(uh3d_prev, v3) * dxOmega)
    L1 = inner(f1, v1) * dxLambda \
        + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + D_area * q1_up * cinlet * v1 * dsLambda(1)
        #%BR:   + D_area * q1 * cinlet * v1 * dsLambda(1)  #%BR: use q1 instead of q1_up


    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

    bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)